In [1]:
import glob
import json
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_context('talk')
from pycorn import pc_res3, pc_uni6
import numpy as np
from scipy.optimize import curve_fit
from scipy.stats import linregress
from scipy.signal import find_peaks
import os
from datetime import date
today = date.today().strftime("%Y%m%d")[2:]
print(today)

250425


In [2]:
def Gaussians(x, Amp, Mean, Sigma):
    '''
    Arbitrary sum of gaussian funtions.
    Amp: vector of amplitude values
    Mean: vector of mean values
    Sigma: vector of sigma values
    Amp, Mean and Sigma must have the same length
    '''
    sum_fnc = []
    for pt in x:
        sum_fnc.append(np.array(Amp) * np.exp(-((pt-np.array(Mean))**2)/(2*np.array(Sigma)**2)))
    
    return np.sum(sum_fnc, axis=1)


def gaussian(x, a1, b1, c1):
    '''
    Single Gaussian function (1 kernel).
    '''
    
    return a1*np.exp((-(x-b1)**2)/(2*c1**2)) 

def gaussians6(x, 
               a1, a2, a3, a4, a5, a6, 
               b1, b2, b3, b4, b5, b6, 
               c1, c2, c3, c4, c5, c6):
    '''
    Explicit mixed Gaussian function (5 kernels).
    '''
    
    return ( a1*np.exp((-(x-b1)**2)/(2*c1**2)) + 
           a2*np.exp((-(x-b2)**2)/(2*c2**2)) + 
           a3*np.exp((-(x-b3)**2)/(2*c3**2)) + 
           a4*np.exp((-(x-b4)**2)/(2*c4**2)) + 
           a5*np.exp((-(x-b5)**2)/(2*c5**2))  + 
           a6*np.exp((-(x-b6)**2)/(2*c6**2))
           )        

def gaussians(x, a1, a2, a3, a4, a5, b1, b2, b3, b4, b5, c1, c2, c3, c4, c5):
    '''
    Explicit mixed Gaussian function (5 kernels).
    '''
    
    return a1*np.exp((-(x-b1)**2)/(2*c1**2)) + a2*np.exp((-(x-b2)**2)/(2*c2**2)) + a3*np.exp((-(x-b3)**2)/(2*c3**2)) + a4*np.exp((-(x-b4)**2)/(2*c4**2)) + a5*np.exp((-(x-b5)**2)/(2*c5**2)) 

In [3]:
def agilent_calibration_biorad_std_fitvoid(path_dextran, path_calib, flowrate, comment="" ):
    '''
    chroma: folder path containing the blue-dextran and protein injections
    '''
    savename = today + "_" + comment
    
    hmw_std = {
        'Thyroglobulin':670000,
        'g-globulin (bovine) ':158000,
        'Ovalbumin (chicken) ':44000,
        'Myoglobin (horse)':17000,
        'Cobalamin(vit b12)':1350,
    }


    dextran = np.loadtxt(path_dextran, delimiter=",")
    dextran[:,0] = dextran[:,0] * flowrate

    v_dex, uv_dex = dextran[:,0] , dextran[:,1]
    void = v_dex[np.argmax(uv_dex)]


    
    print ( f"VOID: {void:1.3f} mL")

    proteins = np.loadtxt(path_calib, delimiter=",")
    proteins[:,0] = proteins[:,0] * flowrate
    v_prot, uv_prot = proteins[:,0], proteins[:,1]

    # plt.plot(dextran[:,0], dextran[:,1])
    # plt.plot(proteins[:,0], proteins[:,1])
    # plt.show()

    # Get peaks by fitting to Gaussian functions
    peak_pos_guess = np.array([void*0.95, #void peak 
                               1.22892861 , 1.33212932, 1.54220434, 1.78828965, 2.12824897,
                               #1.25, 1.35, 1.6, 1.80, 2.15
                              ])
    bounds = np.array( 
                       [ [0, 1100], ]   *len(peak_pos_guess) +     #scale
                       [ [1.0,2.6], ]   *len(peak_pos_guess) +     # mean
                       [ [1e-5, 1.0e-1], ] *len(peak_pos_guess) ,     # std
                         ).T
    p0 = np.hstack([np.array([50]*len(peak_pos_guess) ), np.array(peak_pos_guess) , np.array([0.01]*len(peak_pos_guess) )])
    print(p0.shape)
    print(bounds.shape)
    
    # popt= p0
    popt, pcov = curve_fit(gaussians6, 
                           v_prot, 
                           uv_prot, 
                           p0=p0,
                           bounds=bounds ,
                           # maxfev=1e6
                          )
    print(popt)
    peaks = popt[len(peak_pos_guess)+1:len(peak_pos_guess)+len(peak_pos_guess)]
    print(peaks)
    

    # Plot
    fig, ax = plt.subplots(ncols=2, figsize=(12,5))
    ax[0].plot(v_dex, uv_dex, label='Dextran')
    ax[0].plot(v_prot, uv_prot, 'k-', label='Proteins')
    ax[0].plot(v_prot, gaussians6(v_prot, *popt), 'r-', linewidth=1, label='Fit')
    
    for i in range(len(peak_pos_guess)):
        color='r'
        if i == 0:
            color='gray'
        ax[0].fill_between(v_prot, gaussian(v_prot, *popt[i::len(peak_pos_guess)]), color=color, alpha=0.2)

        
    
    ax[0].set(xlim=[0.8,3.0])
    ax[0].legend(loc='center left', bbox_to_anchor=(1, 0.5) )#loc='best')
    ax[0].set(xlabel='Retention volume / mL', ylabel='A280 / a.u.',)# title=chroma.split('/')[-2])

    
    # Calibration curve
    Kav = (peaks - void) / (3-void)
    log10mw = np.log10(list(hmw_std.values()))
    slope, intercept, rvalue, pvalue, stderr = linregress(log10mw, Kav)
    xs = np.linspace(log10mw.min(), log10mw.max(), 100)
    
    for peak in peaks:
        ax[0].axvline(x=peak, color='gray', zorder=-10, alpha=0.5)
        ax[0].text(peak, 1.2*np.max(uv_prot), f"{peak:2.2f} mL", ha="left", va="center", rotation=60, fontsize=10)
    
    ax[0].axvline(x=void, color='k', zorder=-10, alpha=0.8)
    ax[0].text(void, 1.2*np.max(uv_prot), f"Void\n{void:2.2f} mL", ha="right", va="center", rotation=60, fontsize=10)
    
    ax[0].set_ylim( [-1.0, 1.5 * np.max(uv_prot) ] )
    print(f'uv_prot max: {np.max(uv_prot[150:])}')

    ax[1].scatter(log10mw, Kav, color='white', edgecolor='k')
    ax[1].plot(xs, intercept + slope*xs, 'k-', linewidth=1, label=f'$R^2$ = {rvalue**2:.3f}')
    ax[1].legend(loc='best' ) #loc='best',
    ax[1].set(xlabel='log10(MW)', ylabel='Kav',)# title=chroma.split('/')[-2])
    #plt.set_title("S75 ")
    plt.tight_layout()
    plt.savefig(savename + '.png', dpi=300)
    plt.show()
    
    # Save calibration data.
    calibration_data = {
        'log10mw':log10mw.tolist(),
        'Kav':Kav.tolist(),
        'Vc':3,
        'Vo':void,
        'intercept':intercept,
        'slope':slope
    }
    
    print("saving to: " , os.path.abspath ( savename + f'_void_agilent.json' )  )
    with open(savename + f'_void_agilent.json' , 'w') as f:
        json.dump(calibration_data, f)

In [ ]:
def agilent_calibration_biorad_std_novoid(path_calib, flowrate, comment="" ):
    '''
    chroma: folder path containing the blue-dextran and protein injections
    '''
    savename = today + "_" + comment
    
    hmw_std = {
        'Thyroglobulin':670000,
        'g-globulin (bovine) ':158000,
        'Ovalbumin (chicken) ':44000,
        'Myoglobin (horse)':17000,
        'Cobalamin(vit b12)':1350,
    }


#     dextran = np.loadtxt(path_dextran, delimiter=",")
#     dextran[:,0] = dextran[:,0] * flowrate

#     v_dex, uv_dex = dextran[:,0] , dextran[:,1]
#     void = v_dex[np.argmax(uv_dex)]


    
#     print ( f"VOID: {void:1.3f} mL")

    proteins = np.loadtxt(path_calib, delimiter=",")
    proteins[:,0] = proteins[:,0] * flowrate
    v_prot, uv_prot = proteins[:,0], proteins[:,1]

    # plt.plot(dextran[:,0], dextran[:,1])
    # plt.plot(proteins[:,0], proteins[:,1])
    # plt.show()

    # Get peaks by fitting to Gaussian functions
    peak_pos_guess = np.array(#[void*0.95, #void peak 
                               1.22892861 , 1.33212932, 1.54220434, 1.78828965, 2.12824897,
                               #1.25, 1.35, 1.6, 1.80, 2.15
                              ])
    bounds = np.array( 
                       [ [0, 1100], ]   *len(peak_pos_guess) +     #scale
                       [ [1.0,2.6], ]   *len(peak_pos_guess) +     # mean
                       [ [1e-5, 1.0e-1], ] *len(peak_pos_guess) ,     # std
                         ).T
    p0 = np.hstack([np.array([50]*len(peak_pos_guess) ), np.array(peak_pos_guess) , np.array([0.01]*len(peak_pos_guess) )])
    print(p0.shape)
    print(bounds.shape)
    
    # popt= p0
    popt, pcov = curve_fit(gaussians6, 
                           v_prot, 
                           uv_prot, 
                           p0=p0,
                           bounds=bounds ,
                           # maxfev=1e6
                          )
    print(popt)
    peaks = popt[len(peak_pos_guess)+1:len(peak_pos_guess)+len(peak_pos_guess)]
    print(peaks)
    

    # Plot
    fig, ax = plt.subplots(ncols=2, figsize=(12,5))
    #ax[0].plot(v_dex, uv_dex, label='Dextran')
    ax[0].plot(v_prot, uv_prot, 'k-', label='Proteins')
    ax[0].plot(v_prot, gaussians6(v_prot, *popt), 'r-', linewidth=1, label='Fit')
    
    for i in range(len(peak_pos_guess)):
        color='r'
        if i == 0:
            color='gray'
        ax[0].fill_between(v_prot, gaussian(v_prot, *popt[i::len(peak_pos_guess)]), color=color, alpha=0.2)

        
    
    ax[0].set(xlim=[0.8,3.0])
    ax[0].legend(loc='center left', bbox_to_anchor=(1, 0.5) )#loc='best')
    ax[0].set(xlabel='Retention volume / mL', ylabel='A280 / a.u.',)# title=chroma.split('/')[-2])

    
    # Calibration curve
    Kav = (peaks - void) / (3-void)
    log10mw = np.log10(list(hmw_std.values()))
    slope, intercept, rvalue, pvalue, stderr = linregress(log10mw, Kav)
    xs = np.linspace(log10mw.min(), log10mw.max(), 100)
    
    for peak in peaks:
        ax[0].axvline(x=peak, color='gray', zorder=-10, alpha=0.5)
        ax[0].text(peak, 1.2*np.max(uv_prot), f"{peak:2.2f} mL", ha="left", va="center", rotation=60, fontsize=10)
    
    ax[0].axvline(x=void, color='k', zorder=-10, alpha=0.8)
    ax[0].text(void, 1.2*np.max(uv_prot), f"Void\n{void:2.2f} mL", ha="right", va="center", rotation=60, fontsize=10)
    
    ax[0].set_ylim( [-1.0, 1.5 * np.max(uv_prot) ] )
    print(f'uv_prot max: {np.max(uv_prot[150:])}')

    ax[1].scatter(log10mw, Kav, color='white', edgecolor='k')
    ax[1].plot(xs, intercept + slope*xs, 'k-', linewidth=1, label=f'$R^2$ = {rvalue**2:.3f}')
    ax[1].legend(loc='best' ) #loc='best',
    ax[1].set(xlabel='log10(MW)', ylabel='Kav',)# title=chroma.split('/')[-2])
    #plt.set_title("S75 ")
    plt.tight_layout()
    plt.savefig(savename + '.png', dpi=300)
    plt.show()
    
    # Save calibration data.
    calibration_data = {
        'log10mw':log10mw.tolist(),
        'Kav':Kav.tolist(),
        'Vc':3,
        'Vo':void,
        'intercept':intercept,
        'slope':slope
    }
    
    print("saving to: " , os.path.abspath ( savename + f'_void_agilent.json' )  )
    with open(savename + f'_void_agilent.json' , 'w') as f:
        json.dump(calibration_data, f)

In [ ]:
dex = '/home/srgerb/scripts/cowboy/calibrations/GLaDOS_calibration_s6/dextran/20250225 235454_29_dextran.dx_MWD1A.CSV'
prot = '/home/srgerb/scripts/cowboy/calibrations/GLaDOS_calibration_s6/prot_standards/20250225 234629_28_prot_standards.dx_MWD1C.CSV'
agilent_calibration_biorad_std_fitvoid(dex, prot, 0.35, comment="GLaDOS_S6" )